In [2]:
!pip install torch torchvision torchaudio clip-by-openai pillow


  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you 

ERROR: Cannot install torch and torchvision==0.5.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [3]:
!pip install torch==1.8.0 torchvision==0.9.0

  Using cached torch-1.8.0-cp38-cp38-win_amd64.whl.metadata (23 kB)
  Using cached torchvision-0.9.0-cp38-cp38-win_amd64.whl.metadata (7.9 kB)
   ---------------------------------------- 0.0/190.5 MB ? eta -:--:--
   - -------------------------------------- 6.3/190.5 MB 38.6 MB/s eta 0:00:05
   --- ------------------------------------ 14.9/190.5 MB 40.9 MB/s eta 0:00:05
   ----- ---------------------------------- 24.1/190.5 MB 42.4 MB/s eta 0:00:04
   ------ --------------------------------- 32.5/190.5 MB 41.3 MB/s eta 0:00:04
   -------- ------------------------------- 40.9/190.5 MB 41.3 MB/s eta 0:00:04
   ---------- ----------------------------- 48.8/190.5 MB 40.9 MB/s eta 0:00:04
   ----------- ---------------------------- 56.9/190.5 MB 40.7 MB/s eta 0:00:04
   ------------- -------------------------- 64.5/190.5 MB 39.9 MB/s eta 0:00:04
   --------------- ------------------------ 72.4/190.5 MB 39.8 MB/s eta 0:00:03
   ---------------- ----------------------- 80.5/190.5 MB 39.8 MB/s

In [6]:
import torch
import clip
from PIL import Image
import os
import random
import xml.etree.ElementTree as ET

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Define dataset paths
image_folder = "C:/Users/2003j/Downloads/into_to_ml/chest_reports/split_data/test/images"
report_folder = "C:/Users/2003j/Downloads/into_to_ml/chest_reports/split_data/test/reports"

# Function to clean report text (removes "XXXX XXXX" and trims length)
def clean_text(text):
    if text is None:
        return ""  # Handle NoneType
    text = text.replace("XXXX XXXX", "").strip()  # Remove redacted text
    return text  # No word limit since CLIP will now handle truncation

# Load test reports and corresponding images
test_reports = []
test_images = []

# Parse XML files to extract "Impression" first, then "Findings"
for report_file in os.listdir(report_folder):
    if report_file.endswith(".xml"):
        report_path = os.path.join(report_folder, report_file)
        tree = ET.parse(report_path)
        root = tree.getroot()

        # Extract Impression & Findings separately
        impression_text, findings_text = "", ""
        for abstract_text in root.findall(".//AbstractText"):
            label = abstract_text.get("Label", "").lower()
            cleaned_text = clean_text(abstract_text.text)  # Clean the text

            if "impression" in label:
                impression_text += cleaned_text + " "
            elif "findings" in label:
                findings_text += cleaned_text + " "

        # Combine: "Impression" first, "Findings" next
        full_report = f"{impression_text} {findings_text}".strip()

        # Extract image filenames from <parentImage> tags
        for parent_image in root.findall(".//parentImage"):
            image_id = parent_image.get("id")  # Example: "CXR1091_IM-0062-1001"
            image_filename = image_id + ".png"  # Assuming images are in .png format
            image_path = os.path.join(image_folder, image_filename)

            if os.path.exists(image_path) and full_report:
                test_reports.append(full_report)
                test_images.append(image_filename)
            else:
                print(f"⚠️ Image not found or empty report: {image_filename}")

# Ensure at least 5 samples are available
if len(test_reports) < 5:
    print("❌ Not enough valid reports with images found.")
    exit()

# Randomly sample 5 reports and their corresponding images
sampled_indices = random.sample(range(len(test_reports)), 5)
test_reports = [test_reports[i] for i in sampled_indices]
test_images = [test_images[i] for i in sampled_indices]

# Convert reports to CLIP-friendly format (using truncate=True to avoid errors)
text_inputs = clip.tokenize(test_reports, truncate=True).to(device)

# Run CLIP on each image
for img_file, report_text in zip(test_images, test_reports):
    image_path = os.path.join(image_folder, img_file)
    print(f"📷 Processing Image: {img_file}")

    # Load and preprocess image
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    # Compute similarity scores
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)

        # Normalize features
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # Compute similarity scores
        similarity = (image_features @ text_features.T).squeeze(0)
        best_match_idx = similarity.argmax().item()

    # Display result
    print(f"🔹 Image: {img_file}")
    print(f"✅ Best Matching Report: {test_reports[best_match_idx][:200]}...")  # Print first 200 characters
    print("-" * 50)


📷 Processing Image: CXR1176_IM-0119-2001.png
🔹 Image: CXR1176_IM-0119-2001.png
✅ Best Matching Report: No acute cardiopulmonary findings. .  The cardiomediastinal silhouette and pulmonary vasculature are within normal limits in size. The lungs are clear of focal airspace disease, pneumothorax, or pleur...
--------------------------------------------------
📷 Processing Image: CXR1625_IM-0406-1003.png
🔹 Image: CXR1625_IM-0406-1003.png
✅ Best Matching Report: 1. Cardiomegaly 2. Indistinct vascular margination which may be secondary to bronchovascular crowding however differential diagnosis includes mild pulmonary edema, atypical infection, inflammation  He...
--------------------------------------------------
📷 Processing Image: CXR3455_IM-1677-2001.png
🔹 Image: CXR3455_IM-1677-2001.png
✅ Best Matching Report: No acute cardiopulmonary findings. .  The cardiomediastinal silhouette and pulmonary vasculature are within normal limits in size. The lungs are clear of focal airspace disease, pne

In [7]:
import os
import xml.etree.ElementTree as ET

# Define dataset paths
image_folder = "C:/Users/2003j/Downloads/into_to_ml/chest_reports/split_data/test/images"
report_folder = "C:/Users/2003j/Downloads/into_to_ml/chest_reports/split_data/test/reports"

# List of images to check (from CLIP output)
clip_matched_images = [
    "CXR1176_IM-0119-2001.png",
    "CXR1625_IM-0406-1003.png",
    "CXR3455_IM-1677-2001.png",
    "CXR273_IM-1188-2001.png",
    "CXR270_IM-1168-2001.png"
]

# Store matched reports
matched_reports = {}

# Scan all XML reports
for report_file in os.listdir(report_folder):
    if report_file.endswith(".xml"):
        report_path = os.path.join(report_folder, report_file)
        tree = ET.parse(report_path)
        root = tree.getroot()

        # Check if this XML contains any of the target images
        for parent_image in root.findall(".//parentImage"):
            image_id = parent_image.get("id") + ".png"  # Convert to full filename

            if image_id in clip_matched_images:
                # Extract "Impression" and "Findings" from this report
                impression_text, findings_text = "", ""

                for abstract_text in root.findall(".//AbstractText"):
                    label = abstract_text.get("Label", "").lower()
                    text = abstract_text.text or ""

                    if "impression" in label:
                        impression_text += text.strip() + " "
                    elif "findings" in label:
                        findings_text += text.strip() + " "

                # Save results
                matched_reports[image_id] = {
                    "xml_file": report_file,
                    "impression": impression_text.strip(),
                    "findings": findings_text.strip()
                }

# Print results
for image_id, report_data in matched_reports.items():
    print(f"📌 **Image:** {image_id}")
    print(f"📄 **XML File:** {report_data['xml_file']}")
    print(f"✅ **Impression:** {report_data['impression']}")
    print(f"🔍 **Findings:** {report_data['findings']}")
    print("-" * 50)

# Save results to a file
with open("matched_reports.txt", "w", encoding="utf-8") as f:
    for image_id, report_data in matched_reports.items():
        f.write(f"Image: {image_id}\n")
        f.write(f"XML File: {report_data['xml_file']}\n")
        f.write(f"Impression: {report_data['impression']}\n")
        f.write(f"Findings: {report_data['findings']}\n")
        f.write("-" * 50 + "\n")

print("✅ Matched reports saved to 'matched_reports.txt'.")


📌 **Image:** CXR1176_IM-0119-2001.png
📄 **XML File:** 1176.xml
✅ **Impression:** 1. No acute pulmonary abnormality.
🔍 **Findings:** The lungs and pleural spaces show no acute abnormality. Heart size and pulmonary vascularity within normal limits. .
--------------------------------------------------
📌 **Image:** CXR1625_IM-0406-1003.png
📄 **XML File:** 1625.xml
✅ **Impression:** No acute disease.
🔍 **Findings:** The heart is normal in size. The mediastinum is stable. Atherosclerotic calcifications of the aortic XXXX are present. The lungs are clear.
--------------------------------------------------
📌 **Image:** CXR270_IM-1168-2001.png
📄 **XML File:** 270.xml
✅ **Impression:** No acute cardiopulmonary findings. .
🔍 **Findings:** The cardiomediastinal silhouette and pulmonary vasculature are within normal limits in size. The lungs are clear of focal airspace disease, pneumothorax, or pleural effusion. There are no acute bony findings.
--------------------------------------------------
📌 